<a href="https://colab.research.google.com/github/YanjunLin-Andrie/NLP_SpaCy_eBay/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import all libraries and dependencies

import pandas as pd
import spacy
import json
import random
import numpy as np
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy.tokens import Span
from collections import Counter
from string import punctuation
from random import shuffle
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [ ]:
# Import training dataset
from google.colab import files
# uploaded = files.upload()
#read the file
ttt = pd.read_csv('Train_Tagged_Titles.tsv', on_bad_lines = 'skip', sep = '\t')
ttt = ttt.replace(np.nan, 'Brand', regex=True)
#read the file
lt_sm = pd.read_csv('Listing_Titles_sm.tsv')

In [ ]:
# File i/o
def load_data(file):
  with open(file, 'r', encoding = 'utf-8') as f:
    data = json.load(f)
  return (data)

def save_data(file, data):
  with open(file, 'w', encoding = 'utf-8') as f:
    json.dump(data, f, indent = 4)

## Use entity ruler to create custom NER model and save to file

In [ ]:
# Get all of the unique tags from ttt dataframe and change them to a list format
all_tags = ttt["Tag"].unique().tolist()

# Create an empty list to collect all the patterns of the entity
patterns = []
# Get all of the tag names
for tag in all_tags:
  # Save list of Tokens under the tag name
  items = ttt["Token"].loc[ttt["Tag"] == f"{tag}"].tolist()
  # Loops through created list of Tokens
  for item in items:
    # Adds the new pattern to pattens list
    patterns.append({'label': f'{tag}', 'pattern': item})

In [ ]:
# Create model
def generate_rules(patterns):

  # Create a blank English model
  nlp = spacy.blank('en')
  # Create the entity ruler and add to entity pipeline
  ruler = nlp.add_pipe('entity_ruler')
  # add patterns list to model
  ruler.add_patterns(patterns)

  # Save model with patterns
  nlp.to_disk('./ebay_ner')

# call function
generate_rules(patterns)

## Test on the created NER model

In [ ]:
# Load model
nlp = spacy.load('./ebay_ner')

In [ ]:
# Collecting testing dataset and convert testing data to list format
sentance_list = lt_sm['Title'].tolist()
test = sentance_list[35:45]
test

['Marc Jacobs Black Leather Crossbody Bag ( M0013555-001 ) $ 275.00 NWT',
 'Mimco Echo tote bag - Used but in New condition',
 'Laurel Burch purse hand bag cut out cats geometric 15x9x4 brown',
 "VERSACE JEANS women 's CLUTCH E1VVBBUX_71494",
 'Brighton Handbag Shoulder Bag Tote Raised Flowers Vines Enamel Butterfly',
 '2021 Solid Jacquemus Crocodile Pattern Mini Crossbody Handbag Shoulder Bag',
 'Fossil Small Green Leather Crossbody Shoulder Bag Brass Adjustable w / key Mint',
 'COACH F10909 Soho Black Leather Hobo Shoulder satchel FLAWLESS !',
 '( Velours-Grün ) - Bags4Less Messenger Bag , Velours-Grün ( Green ) - F3151_230 . Fr',
 'Tesco Cute Peter Rabbit Beatrix Potter Shopping Tote Bag New']

In [ ]:
# Pass test data into built nlp model to extract entities and tags
for i in test:
    doc = nlp(i)
    results = []
    for ent in doc.ents:
      results.append([ent.text,ent.label_])
    print(results)

[['Marc', 'Product Line'], ['Jacobs', 'Brand'], ['Black', 'Brand'], ['Leather', 'Material'], ['Crossbody', 'Type'], ['Bag', 'Type'], ['(', 'No Tag'], ['-', 'No Tag'], ['001', 'No Tag'], [')', 'No Tag'], ['$', 'No Tag'], ['NWT', 'No Tag']]
[['tote', 'Brand'], ['bag', 'Brand'], ['-', 'No Tag'], ['Used', 'Obscure'], ['in', 'Obscure'], ['New', 'No Tag'], ['condition', 'No Tag']]
[['Laurel', 'Pattern'], ['Burch', 'Brand'], ['purse', 'Type'], ['hand', 'Brand'], ['bag', 'Obscure'], ['out', 'No Tag'], ['brown', 'Color']]
[['VERSACE', 'Brand'], ['JEANS', 'Brand'], ['women', 'Department'], ["'s", 'Brand'], ['CLUTCH', 'Brand']]
[['Brighton', 'Brand'], ['Handbag', 'Type'], ['Shoulder', 'Handle Style'], ['Bag', 'Brand'], ['Tote', 'Brand'], ['Raised', 'No Tag'], ['Flowers', 'Brand'], ['Vines', 'Pattern'], ['Enamel', 'Brand'], ['Butterfly', 'Pattern']]
[['2021', 'No Tag'], ['Solid', 'No Tag'], ['Crocodile', 'Pattern'], ['Pattern', 'No Tag'], ['Mini', 'Model'], ['Crossbody', 'Brand'], ['Handbag', 'Typ

In [ ]:
# Optional: save tested data
# save_data('./test.json', results)

## Create NER training set

In [53]:
# Load model
nlp = spacy.load('./ebay_ner') 

TRAIN_DATA = []
for item in ttt['Title'].unique().tolist():
  doc = nlp(item)
  entities = []
  for ent in doc.ents:
    entities.append((ent.start_char, ent.end_char, ent.label_)) 
  if len(entities) > 0:
    TRAIN_DATA.append([item, {'entities': entities}]  )
# print(TRAIN_DATA)
# Save training data
save_data('./ebay_training_data.json', TRAIN_DATA)

# Right format, but json file only has one set of data

In [54]:
TRAIN_DATA[0]

['LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas',
 {'entities': [(0, 5, 'Product Line'),
   (6, 13, 'Brand'),
   (14, 20, 'MPN'),
   (21, 28, 'Type'),
   (29, 38, 'Model'),
   (39, 50, 'Color'),
   (51, 57, 'Brand'),
   (58, 69, 'Color'),
   (70, 76, 'Brand')]}]

In [42]:
len(TRAIN_DATA)

3471

In [ ]:
# TRAIN_DATA = []
# def train_data(model, text):
#   for item in text:
#     doc = nlp(item)
#     entities = []
#     for ent in doc.ents:
#       entities.append((ent.text, ent.label_))
#     if len(entities) > 0:
#       TRAIN_DATA = [item, {'entities': entities}]
#       print(TRAIN_DATA)
#   return (TRAIN_DATA)  
    


# # Load model
# nlp = spacy.load('./ebay_ner')  

# # Call function
# train_data(nlp, ttt['Title'].unique().tolist())

# # Save training data
# save_data('./ebay_training_data.json', TRAIN_DATA)

# print(len(TRAIN_DATA))

In [35]:
# TRAIN_DATA = []
# def train_data(model, text):
#   for item in text:
#     doc = nlp(item)
#     for ent in doc.ents:
#       entities = [[ent.text, ent.label_]]
#       for entity in entities:
#         TRAIN_DATA.append(entity)   

# # Load model
# nlp = spacy.load('./ebay_ner')  
# # Call function
# train_data(nlp, ttt['Title'].unique().tolist())
# # Save training data
# save_data('./ebay_training_data.json', TRAIN_DATA)


# print(len(TRAIN_DATA))

38327


## Train an NER model

In [55]:
# Load training dataset
TRAIN_DATA = load_data('./ebay_training_data.json')
TRAIN_DATA

[['LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas',
  {'entities': [[0, 5, 'Product Line'],
    [6, 13, 'Brand'],
    [14, 20, 'MPN'],
    [21, 28, 'Type'],
    [29, 38, 'Model'],
    [39, 50, 'Color'],
    [51, 57, 'Brand'],
    [58, 69, 'Color'],
    [70, 76, 'Brand']]}],
 ['LOUIS VUITTON Petit Noe Drawstring Shoulder Bag Monogram Leather M42226 39SD442',
  {'entities': [[0, 5, 'Product Line'],
    [6, 13, 'Brand'],
    [14, 19, 'Brand'],
    [20, 23, 'Brand'],
    [24, 34, 'Closure'],
    [35, 43, 'Handle Style'],
    [44, 47, 'Brand'],
    [48, 56, 'Brand'],
    [57, 64, 'Trim Material'],
    [65, 71, 'MPN'],
    [72, 79, 'No Tag']]}],
 ['LOUIS VUITTON Damier Azur Pochette Bosphore Shoulder Bag N51112 LV Auth yt523',
  {'entities': [[0, 5, 'Product Line'],
    [6, 13, 'Brand'],
    [14, 20, 'Brand'],
    [21, 25, 'Color'],
    [26, 34, 'Model'],
    [35, 43, 'Model'],
    [44, 52, 'Brand'],
    [53, 56, 'Type'],
    [57, 63, 'MPN'],
    [64, 66, 'MPN']

In [58]:
# Use saved TRAIN_DATA dataset to train a brand new spacy model
def train_spacy(data, iterations):
  TRAIN_DATA = data
  nlp = spacy.blank('en')
  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last = True)
  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])   
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(iterations):
      print('Starting iteration ' + str(itn))
      random.shuffle(TRAIN_DATA)
      losses = {}
      for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # print(text)
        nlp.update([example], drop = 0.2, sgd = optimizer, losses = losses)
      print(losses)
  return(nlp)
TRAIN_DATA = load_data('./ebay_training_data.json')
nlp = train_spacy(TRAIN_DATA, 30)
# nlp.to_disk('./ebay/ebay_ner_model')

Starting iteration 0
{'ner': 24425.538804872896}
Starting iteration 1
{'ner': 20217.51166151509}
Starting iteration 2
{'ner': 18530.201679562815}
Starting iteration 3
{'ner': 17352.000352510673}
Starting iteration 4
{'ner': 16566.957261032403}
Starting iteration 5
{'ner': 15884.705238123359}
Starting iteration 6
{'ner': 15348.434159244578}
Starting iteration 7
{'ner': 14699.122328524763}
Starting iteration 8
{'ner': 14301.653136940295}
Starting iteration 9
{'ner': 13682.097132907971}
Starting iteration 10
{'ner': 13483.701193121738}
Starting iteration 11
{'ner': 13146.150753692182}
Starting iteration 12
{'ner': 12698.658584634957}
Starting iteration 13
{'ner': 12523.001094375588}
Starting iteration 14
{'ner': 12123.478287795844}
Starting iteration 15
{'ner': 11821.19146760415}
Starting iteration 16
{'ner': 11615.373793066909}
Starting iteration 17
{'ner': 11328.212785259013}
Starting iteration 18
{'ner': 11178.251868464113}
Starting iteration 19
{'ner': 10783.084737452542}
Starting ite

In [59]:
# Create a function to get ebay nlp model performance evaluation metrix
def my_evaluate(model, examples):
  # Create Scorer class
  scorer = Scorer() 
  # Create empty list
  example = []
  # Loop through examples
  for input_, annotations in examples: 
    # Predict
    pred = model(input_)
    print(pred, annotations)
    # Create Example instance
    temp = Example.from_dict(pred, annotations)
    # append to example list
    example.append(temp)
  # Get Scores
  scores = scorer.score(example)
  return scores

In [60]:
# Get score of model from my_evaluate
score = my_evaluate(nlp,TRAIN_DATA)
print(score)

4 Pieces Women Leather Handbag Casual Shoulder Bag Tote Messenger Bag Ladies New {'entities': [[0, 1, 'No Tag'], [2, 8, 'No Tag'], [9, 14, 'Department'], [15, 22, 'Material'], [23, 30, 'Type'], [31, 37, 'Occasion'], [38, 46, 'Brand'], [47, 50, 'Type'], [51, 55, 'Type'], [56, 65, 'Type'], [66, 69, 'Brand'], [70, 76, 'Brand'], [77, 80, 'No Tag']]}
Auth LOUIS VUITTON Sac Triangle M52099 Jaune ( Tassili Yellow ) Epi MI1924 Handbag {'entities': [[0, 4, 'No Tag'], [5, 10, 'Brand'], [11, 18, 'Brand'], [19, 22, 'Brand'], [23, 31, 'Pattern'], [32, 38, 'MPN'], [39, 44, 'Color'], [45, 46, 'No Tag'], [47, 54, 'Color'], [55, 61, 'Color'], [62, 63, 'No Tag'], [64, 67, 'Material'], [68, 74, 'No Tag'], [75, 82, 'Type']]}
Vera Bradley Large Traveler Duffel Bag SHORE ENOUGH Luggage NWT {'entities': [[0, 4, 'Obscure'], [5, 12, 'Brand'], [13, 18, 'Pocket Type'], [19, 27, 'Occasion'], [28, 34, 'Type'], [35, 38, 'Type'], [39, 44, 'Model'], [45, 51, 'Brand'], [52, 59, 'Type'], [60, 63, 'No Tag']]}
Authentic 

## Create train, test set to train NER model

In [ ]:
output_path = './ebay'

In [ ]:
import srsly
import typer
import warnings
from tqdm import tqdm
from pathlib import Path
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [ ]:
sentance_list = lt_sm['Title'].tolist()


In [ ]:
train = sentance_list[:4993]
valid = sentance_list[4994:]

In [ ]:
nlp = spacy.blank('en')
def create_training(TRAIN_DATA):
  db = DocBin()
  for text, annot in tqdm(TRAIN_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for label in annot['entities']:
      span = doc.char_span(label = label, alignment_mode = 'contract')
      if span is None:
        print('Skipping entity')
      else:
        ents.append(span)
    doc.ents = ents
    db.add(doc)
  return(db)

In [ ]:
train = create_training(train)
train.to_disk('./ebay/train.spacy')

In [ ]:
valid = create_training(valid)
valid.to_disk('./ebay/valid.spacy')